<a href="https://colab.research.google.com/github/sbertolin/03MAIR-Algoritmos-de-optimizacion/blob/master/AG3/Santiago_Bertol%C3%ADn_AG3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AG3** - Actividad Guiada 3<br>
Santiago Bertolín Martínez <br>
https://github.com/sbertolin/03MAIR-Algoritmos-de-optimizacion/tree/master/AG3

In [2]:
import urllib.request

# Se importa el fichero de datos con el que se trabajará
file = "swiss42.tsp"
urllib.request.urlretrieve("http://elib.zib.de/pub/mp-testdata/tsp/tsplib/tsp/swiss42.tsp", file)

('swiss42.tsp', <http.client.HTTPMessage at 0x7f9d3cdbea58>)

In [3]:
# Se instala la librería para trabajar con el tipo de fichero descargado
!pip install tsplib95

    100% |████████████████████████████████| 1.6MB 14.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/44/c0/34/6f98693a554301bdb405f8d65d95bbcd3e50180cbfdd98a94e
Successfully built networkx
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: networkx 2.2
    Uninstalling networkx-2.2:
      Successfully uninstalled networkx-2.2


In [0]:
import tsplib95
import random
from math import e

# Se carga el conjunto de datos
problem = tsplib95.load_problem(file)

# Nodos
Nodos = list(problem.get_nodes())

# Aristas
Aristas = list(problem.get_edges()) 

In [5]:
# Devuelve el factorial de un numero
def factorial(n):
  if n == 0:
    return 1
  else:
    return n * factorial(n-1)

# Se genera una solucion aleatoria con comienzo en en el nodo 0
def crear_solucion(Nodos):
  solucion = [0]
  for i in range(len(Nodos)-1):
    solucion = solucion + [random.choice(list(set(Nodos) - set({0}) - set(solucion)))]
  return solucion

# Devuelve la distancia entre dos nodos
def distancia(a, b, problem):
  return problem.wfunc(a,b)

# Devuelve la distancia total de una trayectoria 
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] , problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)


# Test para una solución aleatoria
solucion = crear_solucion(Nodos)
distancia_total(solucion, problem)

4911

In [6]:
# Algoritmo de búsqueda aleatoria. Se van probando posibilidades y se queda con la
# mejor encontrada hasta el momento.
def busqueda_aleatoria(problem, N):
  
  Nodos = list(problem.get_nodes())
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  for i in range(N):
    solucion = crear_solucion(Nodos)
    distancia = distancia_total(solucion, problem)
    
    if distancia < mejor_distancia:
      mejor_solucion = solucion
      mejor_distancia = distancia
      
  print("Mejor solución: ", mejor_solucion)
  print("Distancia     : ", mejor_distancia)
  
  return mejor_solucion

# Test búsqueda aleatoria
sol_aleatoria = busqueda_aleatoria(problem, 1000)

Mejor solución:  [0, 34, 8, 5, 20, 32, 14, 11, 31, 37, 35, 36, 25, 41, 10, 16, 33, 2, 6, 23, 39, 27, 4, 19, 15, 26, 13, 12, 9, 22, 21, 30, 18, 17, 3, 7, 38, 1, 29, 40, 24, 28]
Distancia     :  3939


In [0]:
# Generador de soluciones vecinas: 2-opt (intercambiar 2 nodos)
# Si hay N nodos se generan (N-1)x(N-2)/2 soluciones

# Se corrigen las entradas. En clase la función no recibía como parámetro la variable
# problem, que se utilizaba en el cálculo de la distancia total
def genera_vecina(solucion, problem):
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
      distancia_vecina = distancia_total(vecina, problem)
      if distancia_vecina <= mejor_distancia:
        mejor_distancia = distancia_vecina 
        mejor_solucion = vecina
  
  # Se devuelve la mejor solución como en clase, pero también la distancia
  # de esta solución para evitar volver a tener que calcularla
  return mejor_distancia, mejor_solucion

In [8]:
# Algoritmo de búsqueda en el qual se va saltando de mínimo local en mínimo local
# hasta llegar a un número de iteraciones recibido como parámetro.
# Un mínimo local de una nueva iteración no tiene porque ser mejor solución
# que las anteriores
def busqueda_local(problem, N):
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  Nodos = list(problem.get_nodes())
  
  solucion_referencia = crear_solucion(Nodos)
  
  for i in range(N):
    distancia_vecina, vecina = genera_vecina(solucion_referencia, problem)
    
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      
    solucion_referencia = vecina
    
  return mejor_distancia, mejor_solucion

# Test búsqueda local
dist_local, sol_local = busqueda_local(problem, 100)
print("Mejor solución: ", sol_local)
print("Distancia     : ", dist_local)

Mejor solución:  [0, 6, 26, 5, 13, 19, 4, 2, 28, 30, 38, 22, 39, 24, 40, 21, 9, 10, 14, 16, 15, 37, 1, 3, 27, 31, 35, 36, 17, 7, 18, 12, 11, 25, 41, 23, 8, 29, 34, 33, 20, 32]
Distancia     :  1751


In [0]:
# Generador de 1 solucion vecina 2-opt (intercambiar 2 nodos de forma aleatoria)
def genera_vecina_aleatorio(solucion):
  i = random.choice(range(1, len(solucion)) )
  j = random.choice(list(set(range(1, len(solucion))) - {i}))
  vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]
  return vecina

# Devuelve True con una probabilidad: e**(-1*d)/(T*1.0)
# Devuelve False en los otros casos. Probabilidad: 1 - e**(-1*d)/(T*1.0)
def probabilidad(T,d):
  return random.random() <= (e**(-1*d)/(T*1.0))

# Función para bajar la temperatura de forma lineal
def bajar_temperatura(T):
  return T-1

In [15]:
# problem = datos del problema
# T = Temperatura
def recocido_simulado(problem, T):
  
  solucion_referencia = crear_solucion(Nodos)
  distancia_referencia = distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []
  mejor_distancia = 10e100
  
  while T > 0:
    
    # Genera una solución vecina(aleatoria)
    #vecina = genera_vecina_aleatorio(solucion_referencia)
    #distancia_vecina = distancia_total(vecina, problem)
    
    # Genera una solución vecina (la mejor)
    distancia_vecina, vecina = genera_vecina(solucion_referencia, problem)
  
    # Si es la mejor solución de todas se guarda
    if distancia_vecina < mejor_distancia:
      mejor_solucion = vecina
      mejor_distancia = distancia_vecina
      
    # Si la nueva vecina es mejor se cambia y si es peor se cambia según una probabilidad dependiente de T y de |distancia_referencia - distancia_vecina|
    if distancia_vecina < distancia_referencia or probabilidad(T, abs(distancia_referencia - distancia_vecina) ) :
      solucion_referencia = vecina
      distancia_referencia = distancia_vecina
      
    T = bajar_temperatura(T)
    
  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
    
  return mejor_solucion
  
sol_recocido = recocido_simulado(problem, 100)

La mejor solución encontrada es [0, 28, 39, 21, 40, 24, 22, 38, 30, 29, 8, 10, 12, 18, 26, 4, 3, 27, 2, 9, 23, 41, 25, 11, 6, 7, 17, 31, 35, 36, 37, 15, 16, 20, 33, 34, 32, 14, 19, 13, 5, 1]
con una distancia total de 1764


In [0]:
# Establecer una una funcion de probabilidad para 
# añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
def Add_Nodo(problem, H, T):
  Nodos = list(problem.get_nodes())
  return random.choice( list(set(range(1,len(Nodos))) - set(H) ) )

#Incrementar segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total
def Incrementa_Feromona(problem, T, H):
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  
  return T

#Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
#Evapora 0.3 el valor de la feromona, sin que baje de 1 
def Evaporar_Feromonas(T):
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [19]:
#problem = datos del problema
#N = Número de agentes(hormigas)
def hormigas(problem, N):

  #Nodos
  Nodos = list(problem.get_nodes())
  
  #Aristas
  Aristas = list(problem.get_edges())
  
  #Inicializa las aristas con una cantidad inicial de feromonas: 1 
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N):
    #print("\nAgente:", h)
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1):
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h], T )
      Hormiga[h].append(Nuevo_Nodo)
      
    #Incrementa feromonas en esa arista
    T = Incrementa_Feromona(problem, T, Hormiga[h] )

    #print("Feromonas(1)", T)
    
    #Evapora Feromonas
    T = Evaporar_Feromonas(T)
    
  #print("Feromonas(2)", T)

  #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100

  for h in range(N):
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia = distancia_actual
        
  print("Mejor solución: ", mejor_solucion)
  print("Distancia     : ", mejor_distancia)

        
sol_hormigas = hormigas(problem, 1000)

Mejor solución:  [0, 16, 35, 5, 11, 10, 40, 9, 25, 23, 34, 33, 20, 26, 38, 18, 3, 19, 17, 13, 29, 30, 12, 32, 39, 24, 41, 22, 6, 2, 36, 15, 4, 28, 21, 8, 31, 27, 37, 7, 1, 14]
Distancia     :  3913
